# Storing Teletry

Recover data from the previous steps, make sure you've been through them.

In [ ]:
%store -r HM
%store -r endpoint_address 
%store -r home 
%store -r ca_file 
%store -r private_key 
%store -r certificate_file
%store -r payload
%store -r topic_name
%store -r payloads

Create a table

In [ ]:
table_name = f"bloodpressure{HM}"
table_name

In [ ]:
!echo Creating table "{table_name}" this takes a few seconds
table_arn = !aws dynamodb create-table \
  --table-name {table_name} \
  --attribute-definitions \
    AttributeName=name,AttributeType=S \
    AttributeName=time_stamp,AttributeType=N \
  --key-schema AttributeName=name,KeyType=HASH AttributeName=time_stamp,KeyType=RANGE \
  --provisioned-throughput ReadCapacityUnits=1,WriteCapacityUnits=1 \
  --query TableDescription.TableArn \
  --output text
table_arn = table_arn.s
!aws dynamodb wait table-exists --table-name {table_name}
!echo Table created "{table_arn}"

In [ ]:
print(open('iot-trust.json').read())

In [ ]:
role_name = f"bp2dynamodb_role{HM}"
rule_role_arn = !aws iam create-role \
    --role-name {role_name} \
    --assume-role-policy-document file://iot-trust.json \
    --query Role.Arn \
    --output text
rule_role_arn = rule_role_arn.s
rule_role_arn

In [ ]:
import json

In [ ]:
rule_policy = {
    "Version": "2012-10-17",
    "Statement": {
        "Effect": "Allow",
        "Action": "dynamodb:PutItem",
        "Resource": table_arn
    }
}
rule_policy = json.dumps(rule_policy)
rule_policy

In [ ]:
policy_name=f"bp2dynamodb_rule_policy{HM}"
policy_arn = !aws iam create-policy \
  --policy-name '{policy_name}' \
  --policy-document '{rule_policy}' \
  --query Policy.Arn \
  --output text
policy_arn = policy_arn.s
policy_arn

In [ ]:
!aws iam attach-role-policy \
    --role-name {role_name} \
    --policy-arn {policy_arn}

In [ ]:
sql = f"""
SELECT name, 
    time_stamp, 
    sistolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM '{topic_name}'
"""
rule = {
    "ruleDisabled": False,
    "sql": sql,
    "description": "Send blood preasure measurements to DynamoDB",
    "actions": [{
        "dynamoDBv2": {
            "roleArn": rule_role_arn,
            "putItem": {
                "tableName": table_name
            }
        }
    }]
}
rule = json.dumps(rule)
rule

In [ ]:
rule_file=f"{home}/rule.json" 
with open(rule_file, "w") as f:
    f.write(rule)
rule_file

In [ ]:
print(open(rule_file).read())

In [ ]:
rule_name=f"bprule{HM}" 
!aws iot create-topic-rule \
  --rule-name '{rule_name}' \
  --topic-rule-payload file://{rule_file}

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id=f"NotebookCient{HM}"
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(10)
mqtt.connect()
mqtt

In [ ]:
import random
payload = random.choice(payloads)
print(payload)
mqtt.publish(topic_name, payload , QoS = 0)

In [ ]:
mqtt.disconnect()  